In [189]:
import os
import jams

GUITARSET_ROOT = "../data/guitarset"
AUDIO_DIR = os.path.join(GUITARSET_ROOT, "audio_mono-pickup_mix")
ANNOTATION_DIR = os.path.join(GUITARSET_ROOT, "annotation")

In [190]:
jams_files = []

for f in os.listdir(ANNOTATION_DIR):
    full_path = os.path.join(ANNOTATION_DIR, f)
    jams_files.append(full_path)

print("Total JAMS files:", len(jams_files))
print(jams_files[0])

Total JAMS files: 360
../data/guitarset/annotation/03_SS1-100-C#_comp.jams


In [191]:
jam = jams.load(jams_files[0])
print(jam.file_metadata)

{
  "title": "03_SS1-100-C#_comp",
  "artist": "",
  "release": "",
  "duration": 28.80000000000001,
  "identifiers": {},
  "jams_version": "0.3.1"
}


In [192]:
def normalize_chord_annotations(label):
    """
    Normalize GuitarSet chord labels to triads.

    Parameters
    ----------
    label : str
        expected format: root:quality(...)/bass

    Returns
    -------
    str
        root:maj/bass | root:min/bass | root:dim/bass | root:aug/bass | N

    Input format:
        root:quality(...)/bass

    Output format:
        root:maj | root:min | root:dim | root:aug | N
    """

    try:
        root, rest = label.split(":", 1)
    except ValueError:
        return "N"

    rest = rest.lower().strip()
    if rest.startswith("min"):
        q = "min"
    elif rest.startswith("dim"):
        q = "dim"
    elif rest.startswith("aug"):
        q = "aug"
    else:
        q = "maj"

    bass = rest.split("/")[-1]

    return f"{root}:{q}/{bass}"

def extract_chord_annotations(jam, normalized=True):
    """
    Extract performed chord annotations from a GuitarSet JAMS object.

    Parameters
    ----------
    jam : jams.JAMS
        A JAMS object containing GuitarSet annotations.

    normalized : bool, optional
        If True (default), normalize chords to triads (maj/min/dim/aug).
        If False, keep the original chord labels from the JAMS file.

    Returns
    -------
    List[Dict[str, Any]]
        A list of chord segments, where each segment is represented as a dictionary:
            - 'start': float, start time in seconds
            - 'end': float, end time in seconds
            - 'label': str, chord label (normalized or original)
    """
    # Use the second "chord" annotation namespace (performed chords)
    chord_ann = jam.annotations.search(namespace="chord")[1]
    chords = []

    for c in chord_ann:
        label = c.value
        normalized_label = normalize_chord_annotations(label)

        chords.append({
            "start": c.time,
            "end": c.time + c.duration,
            "label": normalized_label if normalized else label
        })

    return chords

def extract_beat_position(jam):
    """
    Extract beat_position annotations from a GuitarSet JAMS object.

    Parameters
    ----------
    jam : jams.JAMS
        A JAMS object containing GuitarSet annotations.

    Returns
    -------
    List[Dict[str, Any]]
        A list of beat_position segments, where each segment is represented as a dictionary:
            - 'time': float, start time in seconds
            - 'duration': float, duration of the beat in seconds
            - 'measure': int, the measure number
            - 'position': int, the position within the measure (in beats)
            - 'time_signature': str, e.g., "4/4", "6/8", "2/2"
    """
    beat_position_ann = jam.annotations.search(namespace="beat_position")
    if not beat_position_ann:
        return []

    beat_position_data = beat_position_ann[0].data
    beat_positions = []

    for bp in beat_position_data:
        value = bp.value
        beat_positions.append({
            "time": bp.time,
            "duration": bp.duration,
            "measure": value['measure'],
            "position": value['position'],
            "time_signature": f"{value['num_beats']}/{value['beat_units']}"
        })

    return beat_positions

In [193]:
dataset_index = []

for jams_path in jams_files:
    jam = jams.load(jams_path)

    basename = os.path.basename(jams_path).replace(".jams", "")
    audio_path = os.path.join(AUDIO_DIR, basename + "_mix" + ".wav")

    if not os.path.exists(audio_path):
        continue

    dataset_index.append({
        "id": basename,
        "audio_path": audio_path,
        "jams_path": jams_path,
        "chords": extract_chord_annotations(jam, True),
        "beat_position": extract_beat_position(jam),
    })

print("Total usable tracks:", len(dataset_index))

Total usable tracks: 360


In [194]:
dataset_index[0]

{'id': '03_SS1-100-C#_comp',
 'audio_path': '../data/guitarset/audio_mono-pickup_mix/03_SS1-100-C#_comp_mix.wav',
 'jams_path': '../data/guitarset/annotation/03_SS1-100-C#_comp.jams',
 'chords': [{'start': 0.0, 'end': 9.6, 'label': 'C#:maj/1'},
  {'start': 9.6, 'end': 14.399999999999999, 'label': 'F#:maj/1'},
  {'start': 14.399999999999999, 'end': 19.2, 'label': 'C#:maj/1'},
  {'start': 19.2, 'end': 21.599999999999998, 'label': 'G#:maj/1'},
  {'start': 21.599999999999998, 'end': 24.0, 'label': 'F#:maj/1'},
  {'start': 24.0, 'end': 28.80000000000001, 'label': 'C#:maj/b3'}],
 'beat_position': [{'time': 0.0,
   'duration': 0.0,
   'measure': 1,
   'position': 1,
   'time_signature': '4/4'},
  {'time': 0.6,
   'duration': 0.0,
   'measure': 1,
   'position': 2,
   'time_signature': '4/4'},
  {'time': 1.2,
   'duration': 0.0,
   'measure': 1,
   'position': 3,
   'time_signature': '4/4'},
  {'time': 1.7999999999999998,
   'duration': 0.0,
   'measure': 1,
   'position': 4,
   'time_signatur